
# Lecture 2: Methods of Data Collection and Data Cleaning Techniques
Date: [Insert Date]  
Duration: 3 hours

## Table of Contents
1. [Introduction](#Introduction)
2. [Methods of Data Collection](#Methods-of-Data-Collection)
   - [Web Scraping](#Web-Scraping)
   - [APIs](#APIs)
   - [Public Datasets](#Public-Datasets)
3. [Data Cleaning Techniques](#Data-Cleaning-Techniques)
   - [Handling Missing Values](#Handling-Missing-Values)
   - [Text Cleaning](#Text-Cleaning)
   - [Audio and Video Preprocessing](#Audio-and-Video-Preprocessing)
4. [Exploratory Data Analysis (EDA)](#Exploratory-Data-Analysis-(EDA))
   - [Basics of Data Visualization](#Basics-of-Data-Visualization)
   - [Statistical Summary](#Statistical-Summary)
5. [Q&A and Discussion](#QnA-and-Discussion)
6. [Conclusion](#Conclusion)

## Introduction

This lecture will focus on the various methods of data collection, including web scraping, APIs, and public datasets, followed by a section on data cleaning techniques, and concluding with an introduction to exploratory data analysis (EDA). We will learn about the tools and considerations for each method, and practice techniques to prepare data for analysis.


## Methods of Data Collection


### Web Scraping
Web scraping is a method used to extract data from websites. This technique is useful when there is a large amount of data on a website that is not readily available in a downloadable format. Python provides several libraries to perform web scraping, the most popular ones being BeautifulSoup and Scrapy.

In this section, we will demonstrate how to scrape the episode data for 'Grey's Anatomy' from a Fandom website using BeautifulSoup and requests. The goal is to collect the following information:

- Number in series
- Number in season
- Title
- Original air date
- U.S. viewers (millions)

**Note**: Web scraping must be performed in compliance with the website's terms of service. Always check the website's `robots.txt` file and terms of service to ensure that you are allowed to scrape their data.


In [2]:

# Required libraries for web scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd



#### Fetching Webpage Content
To scrape data from a web page, we first need to download the page. This is done by making a GET request to the website's server to retrieve the HTML content. Here, we would typically use the `requests` library to make this request. The `requests.get()` method allows us to send an HTTP GET request to the URL of the webpage.


In [3]:

# URL of the page we want to scrape
url = 'https://greysanatomy.fandom.com/wiki/Grey%27s_Anatomy_Episodes'

# Perform the GET request to fetch the HTML content of the page
response = requests.get(url)

# Check the response status code to ensure the request was successful
if response.status_code == 200:
    html_content = response.text
else:
    html_content = "Request failed; cannot fetch the page."



#### Parsing HTML Content
Once we have the HTML content of the page, we can use BeautifulSoup to parse this content and extract the data we need. BeautifulSoup transforms a complex HTML document into a complex tree of Python objects. The two most common objects are:

- `Tag`: Corresponds to an HTML tag in the original document. These are navigable and can be searched.
- `NavigableString`: Represents the text within the tags, which is what we are often trying to extract.


In [4]:

# The following code would parse the HTML content received from the GET request.

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Now 'soup' contains the parsed HTML content ready for data extraction.



#### Extracting Data
After parsing the HTML content, we can use BeautifulSoup to navigate the parse tree and extract the parts of the HTML that contain the data we are interested in. The data is typically contained within HTML tags, and we can use methods like `find_all()` to search for specific tags, or navigate the tree using tag names as attributes.


In [30]:

# The actual scraping logic to extract data from the HTML content
# Here we loop over each table, find all the rows within, and then extract each cell's content.

# Find all the tables on the page
tables = soup.find_all('table')

# We will extract data from the first 19 tables as per the previous example.
# Define a list to store data for all episodes
all_episodes = []

for table in tables[:19]:
    rows = table.find_all('tr')[1:]  # Skip the header row
    for row in rows:
        cells = row.find_all(['th', 'td'])
        if len(cells) == 5:
            episode_data = {
                'No. in series': cells[0].text.strip(),
                'No. in season': cells[1].text.strip(),
                'Title': cells[2].text.strip().strip('"'),  # Remove potential quotes around the title
                'Original air date': cells[3].text.strip(),
                'U.S. viewers (millions)': cells[4].text.strip().split('[')[0]  # Remove reference links
            }
            all_episodes.append(episode_data)

# Convert to a pandas DataFrame
episodes_df = pd.DataFrame(all_episodes)
episodes_df.head()

,No. in series,No. in season,Title,Original air date,U.S. viewers (millions)
0,1,1,A Hard Day's Night,"March 27, 2005",16.25
1,2,2,The First Cut Is the Deepest,"April 3, 2005",17.71
2,3,3,"Winning a Battle, Losing the War","April 10, 2005",17.99
3,4,4,No Man's Land,"April 17, 2005",19.18
4,5,5,Shake Your Groove Thing,"April 24, 2005",17.90



#### Saving Data
After extracting the data and placing it into a structured format, such as a pandas DataFrame, we can save it to a file for later use. DataFrames can be easily exported to various formats including CSV, Excel, JSON, etc. using methods like `to_csv()`, `to_excel()`, etc.


In [31]:

episodes_df.to_csv('episodes_df.csv', index=False)



#### Full code ###

In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = 'https://greysanatomy.fandom.com/wiki/Grey%27s_Anatomy_Episodes'

# Perform the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all the tables on the page
    tables = soup.find_all('table', class_='wikitable')
    
    # We will consider only the first 19 tables as per the instructions
    tables = tables[:19]
    
    # Initialize an empty list to store the data
    episodes_data = []
    
    # Loop through each table and extract the data
    for table in tables:
        # Find all the rows in the table
        rows = table.find_all('tr')
        
        # Skip the header row with column titles
        for row in rows[1:]:  # rows[0] is the header
            # Find all columns for each row
            cols = row.find_all('td')
            
            # Extract text from each column
            no_in_series = cols[0].text.strip()
            no_in_season = cols[1].text.strip()
            title = cols[2].text.strip().replace('"', '')  # Remove quotation marks from title
            original_air_date = cols[3].text.strip()
            us_viewers = cols[4].text.strip().split('[')[0]  # Remove citation brackets
            
            # Append the extracted data to the episodes_data list
            episodes_data.append({
                'No. in series': no_in_series,
                'No. in season': no_in_season,
                'Title': title,
                'Original air date': original_air_date,
                'U.S. viewers (millions)': us_viewers
            })
    
    # Create a DataFrame from the list of episode data
    episodes_df = pd.DataFrame(episodes_data)
    
    # Display the DataFrame
    print(episodes_df.head())  # Display the first few rows
    # The DataFrame can then be saved to a CSV file or used for further analysis
else:
    print(f"Failed to retrieve content, status code: {response.status_code}")



### APIs (Application Programming Interfaces)
An API is a set of rules that allows one software application to interact with another. They define the correct way for a developer to write a program that requests services from an operating system or other application.

In the context of data analysis for media, APIs can be used to collect a vast array of data from social media platforms, video services, and other online sources. A popular video-related API is the YouTube Data API, which provides access to YouTube video and channel data.

**Note**: To use the YouTube Data API, you need to have an API key which can be obtained through the Google Developer Console.


In [9]:

# Libraries required for interacting with the YouTube Data API
import os
from googleapiclient.discovery import build



#### Fetching Data from YouTube Data API
To fetch data from the YouTube Data API, we need to build a service object using the `build` function provided by `googleapiclient.discovery`. This service object is then used to make requests to the API.

Below is a code example that would fetch the most popular videos on YouTube in a specific region and category. The `videos().list()` method is used with the `part='snippet,statistics'` parameter to get basic details about each video along with its statistics.


In [16]:
import json

# You would need to replace 'YOUR_API_KEY' with your actual API key.
api_key = 'My_YouTube_Data_API'
youtube = build('youtube', 'v3', developerKey=api_key)

# This function will fetch the most popular videos in the United States for a specific category
def get_popular_videos(regionCode='US', categoryId='10', maxResults=50):
    response = youtube.videos().list(
        part='snippet,statistics',
        chart='mostPopular',
        regionCode=regionCode,
        videoCategoryId=categoryId,
        maxResults=maxResults
    ).execute()
    
    return response

popular_videos = get_popular_videos()
print(json.dumps(popular_videos, indent=4))


{
    "kind": "youtube#videoListResponse",
    "etag": "HPXMtHeTw_BzsUViBAwEM2LToA0",
    "items": [
        {
            "kind": "youtube#video",
            "etag": "GXrXxIvUwaDRAbuAWZ8yXcLFxQ4",
            "id": "DOZNRoL0310",
            "snippet": {
                "publishedAt": "2023-11-03T04:00:07Z",
                "channelId": "UCKrdjiuS66yXOdEZ_cOD_TA",
                "title": "Megan Thee Stallion - Cobra [Official Video]",
                "description": "The official video for Megan Thee Stallion's \"Cobra\" - OUT NOW!\n\nStream \"Cobra\": https://mts.lnk.to/cobra\n\nDownload on ITunes: https://music.apple.com/us/album/cobra-single/1714895312\n\nText Megan Thee Stallion:\n(832) 210-1202\n\nFollow Megan Thee Stallion:\nhttps://www.instagram.com/theestallion\nhttps://www.tiktok.com/@theestallion\nhttps://twitter.com/theestallion\nhttps://www.facebook.com/theestallionn\nhttps://soundcloud.com/megan-thee-stallion\n\n--\n#MeganTheeStallion #Cobra #OfficialVideo",
            

## Ottenere una chiave API per YouTube

Per ottenere una chiave API per le API di YouTube, devi seguire alcuni passaggi che coinvolgono l'utilizzo della Google Cloud Console. Ecco una guida passo dopo passo:

1. **Accedi a Google Cloud Console**: Visita [Google Cloud Console](https://console.cloud.google.com/) e accedi con il tuo account Google.

2. **Crea un Nuovo Progetto**: Una volta nella console, crea un nuovo progetto cliccando su "Seleziona un progetto" e poi su "Nuovo Progetto". Assegna un nome al tuo progetto e clicca su "Crea".

3. **Seleziona il Progetto Creato**: Assicurati che il progetto appena creato sia selezionato nella console.

4. **Abilita le YouTube Data API**: Vai alla sezione "API e Servizi" > "Libreria", cerca "YouTube Data API v3", e clicca su di essa. Poi clicca su "Abilita" per attivare l'API per il tuo progetto.

5. **Crea le Credenziali**: Dopo aver abilitato le API, vai a "Credenziali" nel menu a sinistra. Clicca su "Crea credenziali" e seleziona "Chiave API". Questo genererà una nuova chiave API.

6. **Restrizioni della Chiave API (Opzionale)**: Dopo aver creato la chiave API, è consigliabile impostare delle restrizioni per migliorare la sicurezza. Puoi limitare l'uso della chiave API a specifiche API (come YouTube Data API) e/o limitare i referrer (indirizzi web) che possono utilizzare la chiave.

7. **Copia la Tua Chiave API**: Una volta generata, copia la tua chiave API. Sarà questa la chiave da utilizzare nelle tue richieste alle API di YouTube.

8. **Conserva la Chiave in Modo Sicuro**: Assicurati di conservare la tua chiave API in un luogo sicuro e non condividerla pubblicamente, in quanto può essere utilizzata per accedere a servizi Google a tuo nome.

Ricorda che l'utilizzo delle API di Google, inclusa quella di YouTube, è soggetto a limiti di quota e, in alcuni casi, a costi. Assicurati di familiarizzare con queste restrizioni per evitare costi inaspettati o interruzioni del servizio. Puoi monitorare e gestire l'uso delle tue API nella Google Cloud Console.

### Public Datasets
The IMDb public datasets are subsets of data from the IMDb database that are available for personal and non-commercial use. They are updated daily and provided in gzipped, tab-separated-values (TSV) format in UTF-8 character set. 
The IMDb datasets can be accessed and downloaded from the following URL: https://datasets.imdbws.com/.

These datasets include:

1. **title.akas.tsv.gz**: Contains alternative titles for movies with details such as the region and language, and attributes like whether it is the original title.

2. **title.basics.tsv.gz**: Includes basic information about titles like type of title, primary and original title, release year, runtime, and genres.

3. **title.crew.tsv.gz**: Lists the directors and writers associated with a title using IMDb's unique identifier system.

4. **title.episode.tsv.gz**: Provides information about TV series episodes, including season and episode number.

5. **title.principals.tsv.gz**: Contains information about the main participants in a title, such as cast and crew, and their roles.

6. **title.ratings.tsv.gz**: Shows the average rating and number of votes for titles.

7. **name.basics.tsv.gz**: Contains information about individuals involved in film and television, including their known-for titles and primary professions.

In [32]:
# import pandas as pd
# import requests
# import gzip
# from io import StringIO

# # Function to download and decompress gzipped datasets
# def download_decompress_imdb_dataset(url):
#     response = requests.get(url)
#     gzip_file = gzip.GzipFile(fileobj=StringIO(response.content))
#     return pd.read_csv(gzip_file, sep='\t', low_memory=False, dtype=str)

# # URLs for the datasets
# episode_url = 'https://datasets.imdbws.com/title.episode.tsv.gz'
# principals_url = 'https://datasets.imdbws.com/title.principals.tsv.gz'

# # Download and decompress the datasets
# title_episode_df = download_decompress_imdb_dataset(episode_url)
# title_principals_df = download_decompress_imdb_dataset(principals_url)

# # Merge the datasets on the title identifier 'tconst'
# merged_df = pd.merge(title_episode_df, title_principals_df, on='tconst')



### Data Cleaning Techniques
Data cleaning is the process of preparing data for analysis by removing or modifying data that is incorrect, incomplete, irrelevant, duplicated, or improperly formatted. This is a fundamental step since the quality of data and the amount of useful information that can be derived from it directly affects the ability of our analysis to be accurate and reliable.

#### Handling Missing Values
Missing data can occur when no information is provided for one or more items or for a whole unit. Missing data is a common issue in data analysis and can be dealt with in several ways:

- **Dropping**: Removing the rows or columns with missing values.
- **Filling**: Replacing missing values with a specific value or a statistic (mean, median, etc.).
- **Interpolation**: Estimating the missing values from other data points.


In [80]:
GA_cleaning_df = pd.read_csv('GA_cleaning.csv')

In [81]:

# First, let's check for missing values in our dataset
missing_values_count = GA_cleaning_df.isnull().sum()
missing_values_count


Season                     129
No. in series              129
No. in season              129
Title_Text                 129
Title_Link                 129
Original air date          129
U.S. viewers (millions)    129
dtype: int64

In [82]:

# If the missing values are not significant, we can choose to drop them
# Here's how to drop rows with any missing values
GA_cleaned_df_drop = GA_cleaning_df.dropna()

# Check the result
GA_cleaned_df_drop.isnull().sum()
GA_cleaning_df.head()

,Season,No. in series,No. in season,Title_Text,Title_Link,Original air date,U.S. viewers (millions)
0,Season 1,1.0,1.0,"""A Hard Day's Night""",https://greysanatomy.fandom.com/wiki/A_Hard_Da...,"March 27, 2005",16.25[1]
1,Season 1,2.0,2.0,"""The First Cut Is the Deepest""",https://greysanatomy.fandom.com/wiki/The_First...,"April 3, 2005",17.71[2]
2,Season 1,3.0,3.0,"""Winning a Battle, Losing the War""",https://greysanatomy.fandom.com/wiki/Winning_a...,"April 10, 2005",17.99[3]
3,Season 1,4.0,4.0,"""No Man's Land""",https://greysanatomy.fandom.com/wiki/No_Man%27...,"April 17, 2005",19.18[4]
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN



#### Cleaning the "U.S. viewers (millions)" Column
The "U.S. viewers (millions)" column is expected to contain numerical values representing the viewership in millions. However, due to the nature of web scraping and the variability of data on web pages, this column may have been imported as a string type and could contain non-numeric characters or missing values. 

To ensure that we can perform numerical operations on this data, we need to clean this column by:
- Converting the column to a numeric type using `pd.to_numeric()`.
- Handling any non-numeric values or strings that cannot be converted to numbers.
- Filling in any remaining missing values with an appropriate statistic such as the mean or median.


In [83]:

# Remove unwanted characters from 'U.S. viewers (millions)' column
# This can be done using the str.replace() method with a regular expression
GA_cleaning_df['U.S. viewers (millions)'] = GA_cleaning_df['U.S. viewers (millions)'].str.extract(r'([\d\.]+)')

# Convert the 'U.S. viewers (millions)' column to numeric
GA_cleaning_df['U.S. viewers (millions)'] = pd.to_numeric(GA_cleaning_df['U.S. viewers (millions)'], errors='coerce')

# Now, df['U.S. viewers (millions)'] is a numeric column

# Check the cleaned 'U.S. viewers (millions)' column
GA_cleaning_df['U.S. viewers (millions)']
GA_cleaning_df.head()

,Season,No. in series,No. in season,Title_Text,Title_Link,Original air date,U.S. viewers (millions)
0,Season 1,1.0,1.0,"""A Hard Day's Night""",https://greysanatomy.fandom.com/wiki/A_Hard_Da...,"March 27, 2005",16.25
1,Season 1,2.0,2.0,"""The First Cut Is the Deepest""",https://greysanatomy.fandom.com/wiki/The_First...,"April 3, 2005",17.71
2,Season 1,3.0,3.0,"""Winning a Battle, Losing the War""",https://greysanatomy.fandom.com/wiki/Winning_a...,"April 10, 2005",17.99
3,Season 1,4.0,4.0,"""No Man's Land""",https://greysanatomy.fandom.com/wiki/No_Man%27...,"April 17, 2005",19.18
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN



### Exploratory Data Analysis (EDA) with pandas-profiling
Exploratory Data Analysis (EDA) is an important step in any data analysis process. It allows us to understand the data, find patterns and anomalies, and form hypotheses based on the features of the data.

`pandas-profiling` is a library that creates a detailed report for a given DataFrame. It's a great tool for EDA because it automatically computes a lot of useful information that would otherwise require a significant amount of manual work. This includes:

- Descriptive statistics for each column
- Correlations between variables
- Histograms of the distributions of variables
- Detection of missing values, and much more.

**Note**: The `pandas-profiling` library needs to be installed in your Python environment. You can install it using `!pip install pandas-profiling` in a Jupyter Notebook cell.


In [84]:

from pandas_profiling import ProfileReport
profile = ProfileReport(GA_cleaning_df, title='Pandas Profiling Report', explorative=True)
profile.to_widgets()
profile.to_file("your_report.html")


PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.4/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.4/u/import-error

## Q&A and Discussion

## Conclusion